## Importing Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten,GlobalAveragePooling2D
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from pathlib import Path
from sklearn.metrics import classification_report,confusion_matrix
plt.rcParams.update({"font.size":20})

## Data Loading

***Loading train-set***

In [3]:
train_data = np.load('GaborDataset/train_data.npz')

In [4]:
x_train = train_data['x'] / 255 #data #normalisation
y_train = train_data['y'] #lables

In [5]:
print(x_train.shape, y_train.shape)

(812, 256, 256, 3) (812, 4)


***Loading test-set***

In [6]:
test_data = np.load('GaborDataset/test_data.npz')

In [7]:
x_test = test_data['x'] / 255 #data #normalisation
y_test = test_data['y'] #lables

In [8]:
print(x_test.shape, y_test.shape)

(167, 256, 256, 3) (167, 4)


In [9]:
class_labels=['Acne', 'benign', 'Dermatitis', 'Eczema']
no_classes = len(class_labels)
print(no_classes)

4


## VGG16

In [10]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))

58889256/58889256 [==============================] - 34s 1us/step


In [11]:
base_model.trainable = False ## Not trainable weights

In [12]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(4, activation='softmax')

In [13]:
model = models.Sequential([base_model,flatten_layer,dense_layer_1,dense_layer_2,prediction_layer])

In [14]:
model.compile(loss= 'categorical_crossentropy', optimizer="adam",metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 50)                1638450   
                                                                 
 dense_1 (Dense)             (None, 20)                1020      
                                                                 
 dense_2 (Dense)             (None, 4)                 84        
                                                                 
Total params: 16,354,242
Trainable params: 1,639,554
Non-trainable params: 14,714,688
_________________________________________________________________


## Model Building

In [ ]:
results = model.fit(x= x_train,y= y_train,batch_size= 10,epochs= 5)

Epoch 1/5
11/82 [===>..........................] - ETA: 5:52 - loss: 1.3977 - accuracy: 0.3727

## Accuracy and Loss Plot-Graphs

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(results.history["accuracy"],color='green', marker='o', linestyle='dashed',linewidth=2, markersize=12)
plt.title("Accuracy Plot Graph",fontsize=20)
plt.legend(["Accuracy"])
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(results.history["loss"],color='red', marker='o', linestyle='dashed',linewidth=2, markersize=12)
plt.title("Loss Plot Graph",fontsize=20)
plt.legend(["Loss"])
plt.show()

In [ ]:
y_pred = model.predict(x_test, batch_size=10)

In [ ]:
print(y_pred)

In [ ]:
y_predict = []
for i in range(0, len(y_pred)):
    y_predict.append(int(np.argmax(y_pred[i])))
len(y_predict)

In [ ]:
print(y_predict)

In [ ]:
y_true = []
for i in range(0, len(y_test)):
    y_true.append(int(np.argmax(y_test[i])))
len(y_true)

In [ ]:
print(y_true)

## Result Analysis

### Classification Report

In [ ]:
print(classification_report(y_true=y_true,y_pred=y_predict))

### Confusion Matrix

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(data=confusion_matrix(y_true=y_true,y_pred=y_predict),
            annot=True,
            fmt=".4g",
            xticklabels=class_labels,
            yticklabels=class_labels,
            cmap="YlGnBu")
plt.show()

## Model Saving

In [ ]:
model_structure = model.to_json()
f = Path("GaborVGG16_model/GaborVGG16_model_structure.json")
f.write_text(model_structure)

In [ ]:
model.save_weights("GaborVGG16_model/GaborVGG16_model_weights.h5")

# Done